In [ ]:
#  Copyright 2023 Google LLC
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

# MPS state preparation

This bloq prepares a quantum state given its representation as a Matrix Product State (MPS).

Refer to https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.95.110503 for the algorithm used.

In [ ]:
from qualtran.bloqs.chemistry.prepare_mps.prepare_mps import PrepareMPS
from qualtran.bloqs.rotations.phase_gradient import PhaseGradientState
from qualtran.bloqs.state_preparation.state_preparation_via_rotation import StatePreparationViaRotations
import quimb
from qualtran import BloqBuilder
from qualtran.drawing import show_bloq
import numpy as np

## Example of use

The first step is to generate the MPS to be implemented. For convenience Quimb is used, but any
other format is also fine as long as it is possible to extract the tensors from the MPS object.
A very important thing to take into account is that the MPS must be in left orthogonal form and the
bond dimensions must be powers of two.

In [ ]:
def c2pol (c):
    # round 0 to eliminate phase
    if abs(c) < 1e-14:
        c = 0
    return f"{abs(c):.3f} ∠ {np.angle(c, deg=True):04.2f}º"


def get_mps (bitlen):
    mps = quimb.tensor.MPS_rand_state(L=bitlen, bond_dim=2**bitlen, dtype=complex)
    mps.compress()
    return mps


quimb.gen.rand.seed_rand(42)
sites = 3
mps = get_mps(sites)
mps.show()

print("\nCoefficients from Quimb object (polar form):")
for i, c in enumerate(mps.to_dense().reshape((-1))):
    print(f"|{i:0{sites}b}>  {c2pol(c)}")

The parameters that the bloq constructor receives are:

* `tensors`: tuple of tensors (in tuple format) that encode the MPS in left canonical form.
        The format in the case of a single site MPS is `(((coef_0, coef_1),),)`. For a two site
        mps the tensor disposition is
        `([bond_dim_0, physical_ind_0], [bond_dim_0, physical_ind_1])` for the first and second
        sites. In a n-site MPS the disposition is `([bond_dim_0, physical_ind_0], ...,
        [bond_dim_{i-1}, bond_dim_i, physical_ind_i], ...,
        [bond_dim_{n-2}, physical_ind_{n-1}])`. For an example of this encoding refer to the
        tutorial.
* `phase_bitsize`: size of the register that is used to store the rotation angles when loading
    the tensor values. Bigger values increase the accuracy of the results but require
    approximately 2 extra ancilla per qubit of phase_bitsize.
* `uncompute`: wether to implement the MPS preparation circuit or its adjoint.
* `internal_phase_gradient`: a phase gradient state is needed for the decomposition. It can be
    either be provided externally if this attribute is set to `False` or internally otherwise.

These arguments must be provided when building the Bloq directly from the raw tensors that form
the MPS, but it is also possible to generate it from a Quimb MatrixProductState object, in which
case the function `from_quimb_mps` is to be used and the MPS object replaces the `tensors` parameter
(all other parameters remain the same).

## Building the Bloq from a Quimb MPS

From te MPS obtained above it is possible to generate a `PrepareMPS` bloq as follows.

In [ ]:
phase_bitsize = 4
mps_prep = PrepareMPS.from_quimb_mps(mps, phase_bitsize)
show_bloq(mps_prep)
show_bloq(mps_prep.decompose_bloq())

From the bloq decomposition above it is possible to see the three gates that correspond to each one
of the tensors in the MPS. Then, using this bloq in a circuit is very simple.

In [ ]:
bb = BloqBuilder()
state = bb.allocate(sites)
state = bb.add(mps_prep, input_state=state)
coefs = bb.finalize(state=state).tensor_contract()

print(f"accuracy: {abs(np.dot(coefs.conj(), coefs))}")

print("\n* Comparison of the original MPS coefficients and circuit result (coefs in polar form):\n")
for i, (compiled, correct) in enumerate(zip(coefs, mps.to_dense().reshape((-1)))):
    print(f"  |{i:0{sites}b}> result: {c2pol(compiled)}     correct: {c2pol(correct)}")

In the case of using an external phase gradient state the circuit looks as follows.

In [ ]:
mps_prep = PrepareMPS.from_quimb_mps(mps, phase_bitsize, internal_phase_gradient=False)
bb = BloqBuilder()
state = bb.allocate(sites)
pg = bb.add(PhaseGradientState(phase_bitsize))
state, pg = bb.add(mps_prep, input_state=state, phase_grad=pg)
bb.add(PhaseGradientState(phase_bitsize).adjoint(), phase_grad=pg)
coefs = bb.finalize(state=state)

## Building the Bloq directly from the tensors

As mentioned in previous sections, it is also possible to build the bloq directly from the tensors
that form the MPS. Just make sure that the dimensions are in the correct order.

In the example below first the tensors are extracted from the MPS object. Then the coefficients are
computed back from the tensors to better illustrate the dimension outlay. The indices `i` and `j`
correspond to virtual or bond dimensions, whereas the `a`, `b` and `c` correspond to physical
indices.

In [ ]:
tensors = [tensor.data for tensor in mps]
contraction_coefs = np.einsum("ia, ijb, jc -> abc", *tensors).reshape((-1))

When building the bloq from tensors, these need to be as type tuple. PrepareMPS provides a method to
cast multidimensional arrays to tuple: `tensor_to_tuple`.

In [ ]:
tuple_tensors = tuple([PrepareMPS.tensor_to_tuple(t) for t in tensors])
mps_prep = PrepareMPS(tuple_tensors, phase_bitsize)

bb = BloqBuilder()
state = bb.allocate(sites)
state = bb.add(mps_prep, input_state=state)
coefs = bb.finalize(state=state).tensor_contract()

print(f"accuracy: {abs(np.dot(coefs.conj(), coefs))}")

print("\n* Comparison of the original MPS coefficients and circuit result (coefs in polar form):\n")
for i, (compiled, correct) in enumerate(zip(coefs, contraction_coefs)):
    print(f"  |{i:0{sites}b}> result: {c2pol(compiled)}     correct: {c2pol(correct)}")

## Using the adjoint

To conclude this tutorial, it is worth noting that PrepareMPS also implements the adjoint. This
is equivalent to un-preparing a state given by an mps. Below its an illustration of this idea,
first the state is prepared directly from the coefficient list by using `StatePreparationViaRotations`,
then it is brought back to the $|0\rangle$ state by applying the adjoint of the MPS preparation bloq.

In [ ]:
coefs = tuple(mps.to_dense().reshape((-1)))
mps_prep_adjoint = PrepareMPS.from_quimb_mps(mps, phase_bitsize, uncompute=True, internal_phase_gradient=False)
state_prep = StatePreparationViaRotations(phase_bitsize, coefs)

bb = BloqBuilder()
state = bb.allocate(sites)
pg = bb.add(PhaseGradientState(phase_bitsize))
state, pg = bb.add(state_prep, target_state=state, phase_gradient=pg)
state, pg = bb.add(mps_prep_adjoint, input_state=state, phase_grad=pg)
bb.add(PhaseGradientState(phase_bitsize).adjoint(), phase_grad=pg)
circuit = bb.finalize(state=state)
coefs = circuit.tensor_contract()

show_bloq(circuit)

print("\nCoefficients from Quimb object (polar form):")
for i, c in enumerate(coefs):
    print(f"|{i:0{sites}b}>  {c2pol(c)}")

Note that the end result does not have to be a unit vector because of the approximation error in
`DecomposeGateViaHR`.